In [43]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np


# Load the housing data into a Pandas DataFrame
df = pd.read_csv("/content/Housing.csv")

# Split the data into training and testing sets
train_data = df.sample(frac=0.9, random_state=0)
test_data = df.drop(train_data.index)

def preprocess(train, test, target='price'):
    # Combine the train and test sets and drop the 'Id' and target columns
    combined_data = pd.concat([train.drop(columns=['area', target]), test.drop(columns=['area'])], axis=0)

    # Select only the numeric features
    numeric_features = combined_data.select_dtypes(include=[np.number]).columns

    # Normalize the numeric features
    combined_data[numeric_features] = (combined_data[numeric_features] - combined_data[numeric_features].mean()) / combined_data[numeric_features].std()
    combined_data[numeric_features] = combined_data[numeric_features].fillna(0)

    # One-hot encode the categorical features
    combined_data = pd.get_dummies(combined_data, dummy_na=True)

    # Split the preprocessed data back into train and test sets
    preprocessed_train = combined_data[:train.shape[0]].copy()
    preprocessed_test = combined_data[train.shape[0]:].copy()

    # Normalize the target column (SalePrice)
    mean, std = train[target].mean(), train[target].std()
    preprocessed_train[target] = (train[target] - mean) / std 

    return preprocessed_train, preprocessed_test

train_data, test_data = preprocess(train_data, test_data)
print(train_data.head())

class HousingModel(nn.Module):
    def init(self):
        super(HousingModel, self).init()
        self.mean = 180921.19589041095
        self.std = 79442.50288288662
        self.unnormalize = False
        self.dropout = False
        self.dropout_layer = nn.Dropout(0.3)
        self.fc1 = nn.Linear(256, 256)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(256, 256)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(256, 128)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(128, 1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        if self.dropout:
            x = self.dropout_layer(x)
            x = self.fc3(x)
            x = self.relu3(x)
            x = self.fc4(x)
            if self.unnormalize:
                x = x * self.std + self.mean
                return x

     bedrooms  bathrooms   stories   parking     price  mainroad_no  \
239  0.047235  -0.569663  0.224204  0.355649 -0.098538            0   
113  0.047235  -0.569663 -0.928544  1.516299  0.696079            0   
325  1.402131  -0.569663  0.224204 -0.805002 -0.404875            0   
66  -1.307661  -0.569663 -0.928544  0.355649  1.145372            0   
479  1.402131  -0.569663  0.224204 -0.805002 -0.971132            1   

     mainroad_yes  mainroad_nan  guestroom_no  guestroom_yes  ...  \
239             1             0             1              0  ...   
113             1             0             1              0  ...   
325             1             0             1              0  ...   
66              1             0             1              0  ...   
479             0             0             1              0  ...   

     airconditioning_no  airconditioning_yes  airconditioning_nan  \
239                   1                    0                    0   
113                 